# Number of Rainy Days

This function should run a SQL query on a dataframe of weather data.  The SQL query should return one column and one row - a count of the number of days in the dataframe where the rain column is equal to 1 (i.e., the number of days it rained).  The dataframe will be titled `weather_data`. You'll need to provide the SQL query.  You might find SQL's `count` function useful for this exercise.  You can read more about it here:
    
https://dev.mysql.com/doc/refman/5.1/en/counting-rows.html
    
You might also find that interpreting numbers as integers or floats may not work initially.  In order to get around this issue, it may be useful to cast these numbers as integers.  This can be done by writing `cast(column as integer)`.

So for example, if we wanted to cast the `maxtempi` column as an integer, we would actually write something like `where cast(maxtempi as integer) = 76`, as opposed to simply `where maxtempi = 76`.

In [7]:
import pandas
import pandasql

def num_rainy_days(filename):
    weather_data = pandas.read_csv(filename)
    q = """
    SELECT COUNT(rain) FROM weather_data WHERE cast(rain as integer) = 1
    """
    rainy_days = pandasql.sqldf(q.lower(), locals())
    return rainy_days

num_rainy_days('weather_underground.csv')

,count(rain)
0,10


# Temp on Foggy and Nonfoggy Days

This function should run a SQL query on a dataframe of weather data.  The SQL query should return two columns and two rows - whether it was foggy or not (0 or 1) and the max `maxtempi` for that fog value (i.e., the maximum max temperature for both foggy and non-foggy days).  The dataframe will be titled `weather_data`. You'll need to provide the SQL query.

In [9]:
def max_temp_aggregate_by_fog(filename):
    weather_data = pandas.read_csv(filename)
    q = """
    SELECT fog, max(maxtempi) FROM weather_data GROUP BY fog
    """
    foggy_days = pandasql.sqldf(q.lower(), locals())
    return foggy_days

max_temp_aggregate_by_fog('weather_underground.csv')

,fog,max(maxtempi)
0,0,86
1,1,81


# Mean Temp on Weekends

This function should run a SQL query on a dataframe of weather data.  The SQL query should return one column and one row - the average `meantempi` on days that are a Saturday or Sunday (i.e., the the average mean temperature on weekends). The dataframe will be titled `weather_data` and you can access the date in the dataframe via the `date` column.

You'll need to provide the SQL query.

Also, you can convert dates to days of the week via the `strftime` keyword in SQL. For example, `cast(strftime('%w', date) as integer)` will return `0` if the date is a Sunday or `6` if the date is a Saturday.

In [14]:
def avg_weekend_temperature(filename):
    weather_data = pandas.read_csv(filename)
    q = """
    SELECT avg(meantempi) FROM weather_data WHERE cast(strftime('%w', date) as integer) IN (0,6)
    """
    mean_temp_weekends = pandasql.sqldf(q.lower(), locals())
    return mean_temp_weekends

avg_weekend_temperature('weather_underground.csv')

,avg(meantempi)
0,65.111111


# Mean Temp on Rainy Days

This function should run a SQL query on a dataframe of weather data. More specifically you want to find the average minimum temperature (`mintempi` column of the weather dataframe) on rainy days where the minimum temperature is greater than 55 degrees.

In [15]:
def avg_min_temperature(filename):
    weather_data = pandas.read_csv(filename)
    q = """
    SELECT avg(mintempi) FROM weather_data WHERE cast(rain as integer) = 1 AND mintempi > 55
    """
    avg_min_temp_rainy = pandasql.sqldf(q.lower(), locals())
    return avg_min_temp_rainy

avg_min_temperature('weather_underground.csv')

,avg(mintempi)
0,61.25


# Fix Turnstile Data

You want to write a function that will update each row in the text file so there is only one entry per row. A few examples below:
```
A002,R051,02-00-00,05-28-11,00:00:00,REGULAR,003178521,001100739
A002,R051,02-00-00,05-28-11,04:00:00,REGULAR,003178541,001100746
A002,R051,02-00-00,05-28-11,08:00:00,REGULAR,003178559,001100775
```

Write the updates to a different text file in the format of `updated_ + filename`. For example:
1. if you read in a text file called `turnstile_110521.txt`
1. you should write the updated data to `updated_turnstile_110521.txt`

The order of the fields should be preserved. 

You can see a sample of the turnstile text file that's passed into the function and the the corresponding updated file by downloading these files from the resources:

1. Sample input file: `turnstile_110528.txt`
1. Sample updated file: `solution_turnstile_110528.txt`

This is the basic code that will get run:

In [65]:
import csv

file_in = open('turnstile_110507.txt', 'r')
file_out = open('updated_turnstile_110507.txt', 'w')

csv_reader = csv.reader(file_in)
csv_writer = csv.writer(file_out)
    
# Keep a list of all the output rows we create
output_rows = []

# Iterate over each row of the CSV file
for row in csv_reader:
    
    # Get the first three items for each row. Using pop()
    # will remove them from the list, leaving a set of
    # values divisible by 5. Each set of five values
    # we wil call a "chunk"
    el1 = row.pop(0)
    el2 = row.pop(0)
    el3 = row.pop(0)
    
    # Figure out how many "chunks" of five elements are left
    number_of_chunks = len(row) / 5
    
    # Iterate over each "chunk" of elements
    for chunk_index in range(0, number_of_chunks):
        start_index = chunk_index * 5
        end_index = start_index + 5
        combined_row = [el1, el2, el3] + row[start_index: end_index]
        output_rows.append(combined_row)

# Write the output
csv_writer.writerows(output_rows)

# Close the file handles that are open
file_in.close()
file_out.close()

In [67]:
def fix_turnstile_data(filenames):

    for filename in filenames:
        
        file_in = open(filename, 'r')
        file_out = open('updated_' + filename, 'w')
        csv_reader = csv.reader(file_in)
        csv_writer = csv.writer(file_out)
    
        # Keep a list of all the output rows we create
        output_rows = []

        # Iterate over each row of the CSV file
        for row in csv_reader:
    
            # Get the first three items for each row. Using pop()
            # will remove them from the list, leaving a set of
            # values divisible by 5. Each set of five values
            # we wil call a "chunk"
            el1 = row.pop(0)
            el2 = row.pop(0)
            el3 = row.pop(0)

            # Figure out how many "chunks" of five elements are left
            number_of_chunks = len(row) / 5

            # Iterate over each "chunk" of five elements. This can be 
            # hard to visualize, it would look something like this:
            # 
            #       Chunk One           Chunk Two
            # ┌──────────────────┐┌──────────────────┐
            # 'a','b','c','d','e','f','g','h','i','j'
            #
            
            for chunk_index in range(0, number_of_chunks):
                start_index = chunk_index * 5
                end_index = start_index + 5
                combined_row = [el1, el2, el3] + row[start_index: end_index]
                output_rows.append(combined_row)

        # Write the output
        csv_writer.writerows(output_rows)        
                
        # Close the files we have open
        file_in.close()
        file_out.close()

# Combining Turnstile Data

Write a function that takes the files in the list `filenames`, which all have the columns `C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn`, and consolidates them into one file located at output_file.  There should be ONE row with the column headers, located at the top of the file. The input files do not have column header rows of their own.
    
For example, if file_1 has:
```
line 1 ...
line 2 ...
```

and another file, file_2 has:
```
line 3 ...
line 4 ...
line 5 ...
```

We need to combine file_1 and file_2 into a master_file like below:
```
C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn
line 1 ...
line 2 ...
line 3 ...
line 4 ...
line 5 ...
```

In [68]:
def create_master_turnstile_file(filenames, output_file):
    with open(output_file, 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename, 'r') as f:
                for line in f.readlines():
                    master_file.write(line)

# Fixing Irregular Data

This function should read the csv file located at `filename` into a pandas dataframe, and filter the dataframe to only rows where the `DESCn` column has the value `REGULAR`.
    
For example, if the pandas dataframe is as follows:
```
,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn
0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
1,A002,R051,02-00-00,05-01-11,04:00:00,DOOR,3144335,1088159
2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177
3,A002,R051,02-00-00,05-01-11,12:00:00,DOOR,3144424,1088231
```

The dataframe will look like below after filtering to only rows where `DESCn` column has the value `REGULAR`:
```
0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177
```

In [69]:
import pandas

def filter_by_regular(filename):
    turnstile_df = pandas.read_csv(filename)
    filtered_df = turnstile_df[turnstile_df['DESCn'] == 'REGULAR']
    return filtered_df

# Get Hourly Entries

The data in the MTA Subway Turnstile data reports on the cumulative number of entries and exits per row.  Assume that you have a dataframe called `df` that contains only the rows for a particular turnstile machine
(i.e., unique `SCP`, `C/A`, and `UNIT`). This function should change these cumulative entry numbers to a count of entries since the last reading (i.e., entries since the last row in the dataframe).
    
More specifically, you want to do two things:
1. Create a new column called `ENTRIESn_hourly`
2. Assign to the column the difference between `ENTRIESn` of the current row  and the previous row. If there is any `NaN`, fill/replace it with 1.
    
You may find the pandas functions `shift()` and `fillna()` to be helpful in this exercise.
    
Examples of what your dataframe should look like at the end of this exercise:
```    
       C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn    EXITSn  ENTRIESn_hourly
0     A002  R051  02-00-00  05-01-11  00:00:00  REGULAR   3144312   1088151                1
1     A002  R051  02-00-00  05-01-11  04:00:00  REGULAR   3144335   1088159               23
2     A002  R051  02-00-00  05-01-11  08:00:00  REGULAR   3144353   1088177               18
3     A002  R051  02-00-00  05-01-11  12:00:00  REGULAR   3144424   1088231               71
4     A002  R051  02-00-00  05-01-11  16:00:00  REGULAR   3144594   1088275              170
5     A002  R051  02-00-00  05-01-11  20:00:00  REGULAR   3144808   1088317              214
6     A002  R051  02-00-00  05-02-11  00:00:00  REGULAR   3144895   1088328               87
7     A002  R051  02-00-00  05-02-11  04:00:00  REGULAR   3144905   1088331               10
8     A002  R051  02-00-00  05-02-11  08:00:00  REGULAR   3144941   1088420               36
9     A002  R051  02-00-00  05-02-11  12:00:00  REGULAR   3145094   1088753              153
10    A002  R051  02-00-00  05-02-11  16:00:00  REGULAR   3145337   1088823              243
...
...
```

In [70]:
def get_hourly_entries(df):
    df['ENTRIESn_hourly'] = df['ENTRIESn'] - df['ENTRIESn'].shift(1)
    df['ENTRIESn_hourly'].fillna(1, inplace=True)
    return df

# Get Hourly Exits

The data in the MTA Subway Turnstile data reports on the cumulative number of entries and exits per row.  Assume that you have a dataframe called `df` that contains only the rows for a particular turnstile machine
(i.e., unique SCP, C/A, and UNIT).  This function should change these cumulative exit numbers to a count of exits since the last reading (i.e., exits since the last row in the dataframe).
    
More specifically, you want to do two things:
1. Create a new column called `EXITSn_hourly`
2. Assign to the column the difference between `EXITSn` of the current row and the previous row. If there is any `NaN`, fill/replace it with `0`.
    
You may find the pandas functions `shift()` and `fillna()` to be helpful in this exercise.
    
Example dataframe below: (several columns omitted for brevity)
```
      Unnamed: 0   C/A  UNIT  ...  ENTRIESn    EXITSn  ENTRIESn_hourly  EXITSn_hourly
0              0  A002  R051  ...   3144312   1088151                0              0
1              1  A002  R051  ...   3144335   1088159               23              8
2              2  A002  R051  ...   3144353   1088177               18             18
3              3  A002  R051  ...   3144424   1088231               71             54
4              4  A002  R051  ...   3144594   1088275              170             44
5              5  A002  R051  ...   3144808   1088317              214             42
6              6  A002  R051  ...   3144895   1088328               87             11
7              7  A002  R051  ...   3144905   1088331               10              3
8              8  A002  R051  ...   3144941   1088420               36             89
9              9  A002  R051  ...   3145094   1088753              153            333
```

In [71]:
def get_hourly_exits(df):
    df['EXITSn_hourly'] = df['EXITSn'] - df['EXITSn'].shift(1)
    df['EXITSn_hourly'].fillna(0, inplace=True)
    return df

# Time to Hour

Given an input variable time that represents time in the format of:
```
"00:00:00" (hour:minutes:seconds)
```
    
Write a function to extract the hour part from the input variable time and return it as an integer. 

For example:
1. if hour is `00`, your code should return `0`
2. if hour is `01`, your code should return `1`
3. if hour is `21`, your code should return `21`
        
Please return hour as an integer.

In [72]:
def time_to_hour(time):
    return int(time.split(':')[0])

# Reformat Subway Dates

The dates in our subway data are formatted in the format `month-day-year`. The dates in our weather underground data are formatted `year-month-day`.    

In order to join these two data sets together, we'll want the dates formatted the same way.  Write a function that takes as its input a date in the MTA Subway data format, and returns a date in the weather underground format.
    
Hint: There are a couple of useful functions in the datetime library that will help on this assignment, called `strptime` and `strftime`. 

In [83]:
from datetime import datetime

def reformat_subway_dates(date_string):
    date = datetime.strptime(date_string, '%m-%d-%y')
    date_formatted = date.strftime('%Y-%m-%d')
    return date_formatted

reformat_subway_dates('05-01-2000')

'2000-05-01'